In [1]:
import numpy as np
import pandas as pd
import ast
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
#from google.colab import drive
#drive.mount('/content/drive')

import os
os.chdir("/content/drive/My Drive")
!ls

 churn-bigml-20.csv			     IRIS.csv
 climate_2009_2016.csv			    'Loan Prediction.csv'
'Colab Notebooks'			     Mall_Customers.csv
 Cuisine_rating.csv			     model.pkl
'Data_Science_Week_8_Assignment (3).ipynb'   models
 df_original.csv			     streamlit_app
 df_scaled.csv				     tmdb_5000_credits.csv
 Diabetes-Classification.csv		     tmdb_5000_movies.csv
 fitness_class_2212.csv			     twitter_training.csv
 Foreign_Exchange_Rates.xls		     weather_data_extended.csv
 heart.csv				    'Week 8.ipynb'


In [3]:
#Load the datasets
movie = pd.read_csv('tmdb_5000_movies.csv')
credits = pd.read_csv('tmdb_5000_credits.csv')

#Merge both datasets on title
movies = movie.merge(credits, on = 'title')

#Keep important columns

movies = movies[['id', 'title', 'overview', 'keywords', 'genres', 'cast', 'crew']]

#Drop rows with null values
movies.dropna(inplace = True)

/tmp/ipython-input-1916789793.py:3: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262

In [4]:
#Process 'overview' column
movies['overview'] = movies['overview'].apply(lambda x: x.split())

# Process 'keywords' column

def convert_keywords(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

movies['keywords'] = movies['keywords'].apply(convert_keywords)

In [5]:
#Process 'genres' column

def convert_genres(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

movies['genres'] = movies['genres'].apply(convert_genres)

In [6]:
#Process 'cast' column, top 3 only

def extract_cast(obj):
    l = []
    count = 0
    for i in ast.literal_eval(obj):
        if count != 3:
            l.append(i['name'])
            count += 1
        else:
            break
    return l

movies['cast'] = movies['cast'].apply(extract_cast)

In [8]:
#Process 'crew' column, get the director only

def extract_director(obj):
    l = []
    try:
        crew_list = ast.literal_eval(obj)
    except:
        return l               # return empty list if parsing fails

    for i in crew_list:
        if i.get('job') == 'Director':
            l.append(i.get('name'))
            break
    return l

movies['crew'] = movies['crew'].apply(extract_director)

#Create 'tags' column by combining overview + keywords + genres + cast + crew
movies['tags'] = movies['overview'] + movies['cast'] + movies['crew'] + movies['keywords']

#Final dataset with relevant columns

movies = movies[['id', 'title', 'tags']]

In [9]:
# Remove spaces from tags

movies['tags'] = movies['tags'].apply(lambda x: [i.replace(" ", "") for i in x])

# Stemming

ps = PorterStemmer()

def stemming(text):
    l = []
    for i in text:
        l.append(ps.stem(i))
    return " ".join(l)

movies['tags'] = movies['tags'].apply(stemming)

In [11]:
#Before we go any further, here is a look at the shape of our data frame
movies.head()

,id,title,tags
0,19995,Avatar,"in the 22nd century, a parapleg marin is dispa..."
1,285,Pirates of the Caribbean: At World's End,"captain barbossa, long believ to be dead, ha c..."
2,206647,Spectre,a cryptic messag from bond’ past send him on a...
3,49026,The Dark Knight Rises,follow the death of district attorney harvey d...
4,49529,John Carter,"john carter is a war-weary, former militari ca..."


In [12]:
print(movies.shape)
movies.info()

(1489, 3)
<class 'pandas.core.frame.DataFrame'>
Index: 1489 entries, 0 to 1491
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      1489 non-null   int64 
 1   title   1489 non-null   object
 2   tags    1489 non-null   object
dtypes: int64(1), object(2)
memory usage: 46.5+ KB


In [15]:
movies['tags'].apply(len).describe()

,tags
count,1489.000000
mean,423.932841
std,151.598932
min,123.000000
25%,311.000000
50%,410.000000
75%,511.000000
max,1138.000000


In [16]:
#We can also use 'Counter' to see which words are most common before we do vectorizing
from collections import Counter
Counter(" ".join(movies['tags']).split()).most_common(20)

[('the', 4602),
 ('a', 3083),
 ('to', 2616),
 ('and', 2265),
 ('of', 2115),
 ('hi', 1291),
 ('in', 1268),
 ('is', 997),
 ('with', 735),
 ('he', 640),
 ('an', 610),
 ('on', 591),
 ('for', 563),
 ('that', 546),
 ('as', 512),
 ('when', 487),
 ('her', 476),
 ('by', 465),
 ('their', 458),
 ('from', 436)]

In [17]:
# Vectorization
vectorizer = CountVectorizer(max_features=500, stop_words='english')
vectors = vectorizer.fit_transform(movies['tags']).toarray()

# Cosine similarity
similarity = cosine_similarity(vectors)

In [18]:
# Recommendation function
def Recommendation_system(movie_title):
    # Handle case where movie isn't in the dataset
    if movie_title not in movies['title'].values:
        print("Movie not found in the database!")
        return []

    movie_index = movies[movies['title'] == movie_title].index[0]
    distances = sorted(list(enumerate(similarity[movie_index])),
                       reverse=True, key=lambda x: x[1])

    #extract top 19 similar movie titles (skip the movie itself at index 0)
    recommended_titles = [movies.iloc[i[0]].title for i in distances[1:20]]

    #print the recommendations
    for title in recommended_titles:
        print(title)

    return recommended_titles


In [19]:
Recommendation_system('Avatar')

Independence Day
Aliens vs Predator: Requiem
Beowulf
The Thing
Titan A.E.
Treasure Planet
Edge of Tomorrow
Meet Dave
Aliens in the Attic
Predators
Tears of the Sun
Galaxy Quest
Battleship
Home
The Watch
Prometheus
We Were Soldiers
The 5th Wave
Mission to Mars


In [22]:
#Save the model and data
pickle.dump(movies, open('movies.pkl', 'wb'))
pickle.dump(similarity, open('similarity.pkl', 'wb'))

In [23]:
from flask import Flask, request, jsonify
import pickle

app = Flask(__name__)

#load model artifacts
movies = pickle.load(open('movies.pkl', 'rb'))
similarity = pickle.load(open('similarity.pkl', 'rb'))

def Recommendation_system(movie_title):
    if movie_title not in movies['title'].values:
        return []

    movie_index = movies[movies['title'] == movie_title].index[0]
    distances = sorted(list(enumerate(similarity[movie_index])),
                       reverse=True, key=lambda x: x[1])

    recommended_titles = [movies.iloc[i[0]].title for i in distances[1:20]]
    return recommended_titles

@app.route('/')
def home():
    return "Movie Recommender is running. Use /recommend?title=MovieName"

@app.route('/recommend')
def recommend():
    title = request.args.get('title')
    if not title:
        return jsonify({"error": "No title provided"}), 400

    recs = Recommendation_system(title)
    if not recs:
        return jsonify({"error": "Movie not in dataset"}), 404

    return jsonify({"recommendations": recs})

if __name__ == '__main__':
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)


In [ ]:
#This can now be run locally using app.py